In [1]:
CODE_DIR := "/Users/sal/gap/";;

In [2]:
Read(Concatenation(CODE_DIR, "pmatmul.g"));

# Simple speedups for unsophisticated users on multicore workstations

## Multiplying matrices

We create a couple of random integer matrices and check GAP's default serial multiply.

In [5]:
m1 := RandomMat(400, 400, Rationals);;
m2 := RandomMat(400, 400, Rationals);;
ShowBench(\*, m1, m2);

wall time: 6.73s cpu time: 6.10s memory allocated: 1006.98MB result returned


In a few lines of code, we can implement a simple blocked matrix multiply based on $$(A*B)_{ik} = \sum_j A_{ij}*B_{jk}$$  

In [6]:
MatMulWithTasks := function(m1, m2, chop1, chop2, chop3)
    local  A, B, prodtasks, sumtasks, C;
    
    # divide matrices into blocks
    A := block(m1, chop1, chop2);
    B := block(m2, chop2, chop3);

    # Start chop1*chop2*chop3 multiply tasks
    prodtasks := List([1..chop1], i-> List([1..chop2], j-> 
        List([1..chop3], k -> RunTask(\*, A[i][j],B[j][k]))));
    # And chop1 * chop3 tasks to do the summations
    sumtasks := List([1..chop1], i -> List([1..chop3], k-> 
        RunTask(Accumulate,AddMat,ShallowCopyMat, 
                prodtasks[i]{[1..chop2]}[k])));
    # Finally wait for the summations to complete and assemble the result
    C := List(sumtasks, row -> List(row, TaskResult));
    return unblock(C);
end;


function( m1, m2, chop1, chop2, chop3 ) ... end

In [6]:
ShowBench(MatMulWithTasks, m1, m2, 4, 4, 4);

wall time: 2.59s cpu time: 15.39s memory allocated: 454.80MB result returned


The `Accumulate` function takes a list of tasks and combines their results as they become available, allowing memory to be recovered.

In [8]:
Display(Accumulate);

function ( op, makebase, tasks )
    local i, acc;
    i := WaitAnyTask( tasks );
    acc := makebase( TaskResult( tasks[i] ) );
    Remove( tasks, i );
    while Length( tasks ) > 0 do
        i := WaitAnyTask( tasks );
        op( acc, TaskResult( tasks[i] ) );
        Remove( tasks, i );
    od;
    return acc;
end


## A Simple Search

We search for *Association Schemes* preserved by interesting permutation groups. Our initial filter selects relevant groups where the problem is non-trivial from GAP's primtive groups database. 

In [5]:
c := cands([136,165],[1..13]);; List(c, x -> x.g); List(c, x-> x.rank);

[ PSL(2, 17), M_11 ]

[ 11, 7 ]

We apply, for now, a brute force search over all partitions of the set $\{2\ldots r\}$ where $r$ is the rank of the permutation action. This is a fair sized search space and grows very rapidly with $r$.

In [7]:
NrPartitionsSet([2..11]);

115975

In [8]:
Display(Brute);

function ( s )
    return Filtered( PartitionsSet( [ 2 .. s.rank ] ), function ( p )
            return TestPartition( s, p );
        end );
end


In [10]:
ShowBench(Brute, c[1]);

wall time: 34.92s cpu time: 29.13s memory allocated: 16.38GB result returned


A very simple approach to parallelising this brute force search produces a useful speedup

In [11]:
Display(ParBrute);

function ( s )
    return ParFiltered( PartitionsSet( [ 2 .. s.rank ] ), function ( p )
            return TestPartition( s, p );
        end, 100 );
end


In [6]:
ShowBench(ParBrute, c[1]);

wall time: 21.15s cpu time: 71.66s memory allocated: 14.43GB result returned


Other things -- some of the atomic data structures?